In [ ]:
!sudo apt-get install tesseract-ocr
!wget https://github.com/tesseract-ocr/tessdata/raw/main/ben.traineddata
!sudo mv ben.traineddata /usr/share/tesseract-ocr/4.00/tessdata/
!sudo apt-get install poppler-utils
!pip install pytesseract -q
!pip install pdf2image -q

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (9,452 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
import pytesseract
from pdf2image import pdfinfo_from_path, convert_from_path
import os

def extract_text_from_scanned_pdf(pdf_path, lang="eng+ben", save_folder=None, batch_size=8):

    filename = pdf_path.split('/')[-1].split('.')[0] + '.txt'

    if save_folder is not None:
        save_folder = os.path.join(save_folder, filename)
    else:
        save_folder = filename

    # Initialize an empty file to write the output
    with open(save_folder, 'w', encoding='utf-8') as f:
        pass  # Just creating the file or clearing if it exists

    # Get the total number of pages in the PDF
    pdf_info = pdfinfo_from_path(pdf_path, userpw=None, poppler_path=None)
    total_pages = pdf_info["Pages"]

    # Process pages in batches to reduce memory usage
    for i in range(0, total_pages, batch_size):
        # Load a batch of pages
        pages = convert_from_path(pdf_path, first_page=i+1, last_page=min(i+batch_size, total_pages))
        print(f"Pages: {i+1}-{min(i+batch_size, total_pages)}")

        for page in pages:
            # Extract text from the page
            text = pytesseract.image_to_string(page, lang=lang)

            # Append the extracted text to the output file
            with open(save_folder, 'a', encoding='utf-8') as f:
                f.write(text)
                f.write("\n\n")  # Optionally add a separator between pages

            # Delete the page object to free up memory
            del page

        # Force garbage collection to free up memory
        import gc
        gc.collect()

    print(f"Text extraction completed. Output saved to {save_folder}")

pdf_path = '/content/drive/MyDrive/Books/PDF/জাসদের উথান পতনঃ অস্থির সময়ের রাজনীতি.pdf'
txt_save_path = '/content/drive/MyDrive/Books/TXT (Eng+Ben)'
extract_text_from_scanned_pdf(pdf_path, save_folder=txt_save_path)
